In [5]:
import os
import math
import torch
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn, Tensor
import torch.utils.data as data_utils
from functools import partial, reduce

In [6]:
from utils.metrics import mse, mae, smape
from models.FFNN import FFNN
from models.LSTM_FFNN import LSTM_FFNN
from models.LSTM_Seq2Seq import LSTM_Seq2Seq
from models.LSTM_Seq2Seq_Att import LSTM_Seq2Seq_Att
from data.data_processing import data_processing
from data.data_transforms import data_transform_std
from data.data_splitting import make_input_output_sequences, train_test_split, shift_sequence

In [7]:
learning_rate = 0.032
lags = 4
h = [4]
test_ratio = 0.3
file_name = 'datasets\covid\OWID_weekly.csv'
features = ['date','new_deaths','icu_patients','hosp_patients']
batch_size = 32
epochs = 100
patience = 5
lstm_hidden_size = 32
lstm_layers = 1
tf = False
FFNN_hidden_1 = 32
FFNN_hidden_2 = 64
random_seed = 2021
tf_ratio = 0.3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
models = ['FFNN', 'LSTM_FFNN', 'LSTM_Seq2Seq', 'LSTM_Seq2Seq_Att']

In [8]:
for m in models:
    forecast_matrix = pd.DataFrame(columns = ['date'])
    print('Model: '+m)
    for horizons in h:
        random.seed(random_seed)
        np.random.seed(random_seed)
        torch.manual_seed(random_seed)
        data, observed = data_processing(file_name, features)
        scalers, df = data_transform_std(data, test_ratio)
        x, y = make_input_output_sequences(data.values, lags, horizons, True)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_ratio)
        if(m == 'FFNN'):
            model = FFNN(FFNN_hidden_1, FFNN_hidden_2, lags, horizons, len(features)-1).to(device)
        elif(m == 'LSTM_FFNN'):
            model = LSTM_FFNN(lstm_hidden_size, lstm_layers, lags, horizons, len(features)-1).to(device)
        elif(m == 'LSTM_Seq2Seq'):
            model = LSTM_Seq2Seq(lstm_hidden_size, lstm_layers, lags, horizons, len(features)-1).to(device)
        elif(m == 'LSTM_Seq2Seq_Att'):
            model = LSTM_Seq2Seq_Att(lstm_hidden_size, lstm_layers, lags, horizons, len(features)-1).to(device)
        for w in range(x_test.shape[0]):
            n_epochs_stop = patience
            epochs_no_improve = 0
            early_stop = False
            min_val_loss = np.Inf
            results = pd.DataFrame(columns = ['date'])
            y_test_dates = pd.DataFrame(y_test[0,:,0])
            x_train_tensor = torch.from_numpy(np.array(x_train[:,:,1:], dtype=np.float32)).float().to(device)
            y_train_tensor = torch.from_numpy(np.array(y_train[:,:,1:], dtype=np.float32)).float().to(device)
            x_test_tensor = torch.from_numpy(np.array(x_test[:,:,1:], dtype=np.float32)).float().to(device)
            y_test_tensor= torch.from_numpy(np.array(y_test[:,:,1:], dtype=np.float32)).float().to(device)
            results['date'] = y_test_dates 
            loss_fn = torch.nn.MSELoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
            train = data_utils.TensorDataset(x_train_tensor, y_train_tensor)
            train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False)
            for epoch in range(epochs):
                train_losses = []
                model.train()
                for x_t, y_t in train_loader:
                    optimizer.zero_grad()
                    y_pred = model(x_t, y_t, tf)
                    loss = loss_fn(y_pred, y_t[:,:,0])
                    loss.backward()
                    optimizer.step()
                    train_losses.append(loss.item())
                train_loss = np.average(train_losses)
                if train_loss < min_val_loss:
                    epochs_no_improve = 0
                    min_val_loss = train_loss
                else:
                    epochs_no_improve += 1
                if epochs_no_improve == n_epochs_stop:
                    break
            with torch.no_grad():
                model.eval()
                y_test_pred = model(x_test_tensor, y_test_tensor, False)
                results['window'+str(w)] = y_test_pred[0,:].cpu().flatten().numpy()
            x_train, x_test, y_train, y_test = shift_sequence(x_train, y_train, x_test, y_test, 1, True)
            if(w==0):
                forecast_matrix = pd.merge(forecast_matrix, results, on=['date'], how='right')
            else:
                forecast_matrix = pd.merge(forecast_matrix, results, on=['date'], how='outer')
        forecast_matrix.head()
        date = pd.DataFrame(forecast_matrix['date'])
        forecast_matrix.set_index('date', inplace=True)
        scaler = scalers['scaler_new_deaths']
        forecast_matrix=scaler.inverse_transform(forecast_matrix)
        forecast_matrix =pd.DataFrame(forecast_matrix)
        forecast_matrix['date'] = date
        forecast_matrix.set_index('date', inplace=True)
        start = 0
        end = 4
        weekly_predictions = []
        window = 0
        for i in range(len(forecast_matrix.columns)-3):
            forecast_matrix_temp = forecast_matrix.iloc[start:end]
            date = forecast_matrix_temp.tail(1).index.item()
            last_row = forecast_matrix_temp.tail(1)
            weekly_predictions.append([date, last_row[window][0], last_row[window+1][0],
                                       last_row[window+2][0],last_row[window+3][0]])
            window = window + 1
            start = end
            end = end+1
        weekly_predictions = pd.DataFrame(weekly_predictions, columns=[ 'date', str('h4_'+m), str('h3_'+m), str('h2_'+m),
                                                                      str('h1_'+m)])
        observed.reset_index(inplace=True, drop= True)
        matrix = reduce(lambda x,y: pd.merge(x,y, on='date'), [observed, weekly_predictions])
        matrix.reset_index(inplace=True, drop= True)
        metrics = np.empty([4, 3])
        for idx, j in enumerate(['h1','h2','h3','h4']):
            mse_metric = ('%.3f' %mse(matrix['new_deaths'], matrix[j+'_'+str(m)]))
            mae_metric = ('%.3f' %mae(matrix['new_deaths'], matrix[j+'_'+str(m)]))
            smape_metric = ('%.3f' %smape(matrix['new_deaths'], matrix[j+'_'+str(m)]))
            print('horizon: '+str(j)+' mse: '+ str(mse_metric)+' mae: '+
                  str(mae_metric)+' smape: '+ str(smape_metric))
            metrics[idx,:] = (mse_metric, mae_metric, smape_metric)
        print("Average: "+"   mse: "+str(('%.3f' %np.average(metrics[:,0]))) + 
              " mae: "+ str(('%.3f' %np.average(metrics[:,1]))) +" smape: "+
              str(('%.3f' %np.average(metrics[:,2]))))
        print("")

Model: FFNN
horizon: h1 mse: 322671.061 mae: 408.254 smape: 17.892
horizon: h2 mse: 382877.510 mae: 476.075 smape: 20.513
horizon: h3 mse: 535571.949 mae: 560.894 smape: 22.476
horizon: h4 mse: 729142.080 mae: 616.084 smape: 23.794
Average:    mse: 492565.650 mae: 515.327 smape: 21.169

Model: LSTM_FFNN
horizon: h1 mse: 147808.785 mae: 296.249 smape: 11.446
horizon: h2 mse: 229266.735 mae: 395.366 smape: 15.521
horizon: h3 mse: 272312.155 mae: 415.892 smape: 16.457
horizon: h4 mse: 558367.757 mae: 614.686 smape: 23.456
Average:    mse: 301938.858 mae: 430.548 smape: 16.720

Model: LSTM_Seq2Seq
horizon: h1 mse: 175802.570 mae: 330.441 smape: 13.065
horizon: h2 mse: 179776.645 mae: 334.958 smape: 13.300
horizon: h3 mse: 195494.321 mae: 366.033 smape: 14.480
horizon: h4 mse: 369758.695 mae: 453.123 smape: 17.133
Average:    mse: 230208.058 mae: 371.139 smape: 14.494

Model: LSTM_Seq2Seq_Att
horizon: h1 mse: 168351.011 mae: 340.745 smape: 13.155
horizon: h2 mse: 174475.249 mae: 330.567 sma